In [ ]:
# SPDX-FileCopyrightText: 2022 Aleksander Grochowicz
#
# SPDX-License-Identifier: GPL-3.0-or-later
import copy

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from itertools import product

from utilities import (sim_no_flex, sim_trans, sim_storage, sim_full_flex)

from multiprocessing import Pool


# Configuration

In [ ]:
# Simulation
solvers = 64 # for parallelisation
sample_length = 100
batch_size = 100

In [ ]:
# Set-up

## Load capacity factors and load data.
cf_ts = pd.read_csv(f"./data/processing/norway_regions_production_corr_{sample_length}y.csv", index_col=0)
dem_ts = pd.read_csv(f"./data/processing/demand_norway_{sample_length}y.csv", index_col=0)/24 # Convert to hourly averages
dem_ts *= 0.128 # calculation based on NVE, accessed 22/11/2022, and Nordpool data

## Capacities
x_initial = np.array([3257.,1811.]) # NVE, accessed 22/11/2022
#storage_cap = np.array([29.9*1e6, 57.3*1e6])
storage_cap = np.array([15000.,30000.]) # enough to cover the max wind demand for more than two weeks
generation_cap = np.array([900.,900.]) # should be comparable to transmission
transmission_cap = 900. # Statnett, accessed 22/11/2022

## Efficiencies
eff_c = 0.75 # PyPSA-Eur efficiency for pumped hydro storage (charge)
eff_d = 0.9 # PyPSA-Eur efficiency for usual hydro (discharge)

# Standard:
# x_initial = np.array([3257.,1811.]) # NVE, accessed 22/11/2022
# storage_cap = np.array([15000.,30000.]) # enough to cover the max wind demand for more than two weeks
# generation_cap = np.array([900.,900.]) # should be comparable to transmission
# transmission_cap = 900. # Statnett, accessed 22/11/2022

# ## Efficiencies
# eff_c = 0.75 # PyPSA-Eur efficiency for pumped hydro storage (charge)
# eff_d = 0.9 # PyPSA-Eur efficiency for usual hydro (discharge)

In [ ]:
# Parallel simulation
##  Parallelisation functions
def parsim_no_flex(x_initial, batch_size, transmission_cap, storage_cap, generation_cap):
    penalties, _ = sim_no_flex(x_initial, batch_size, cf_ts, dem_ts)
    return pd.concat(penalties).sum(axis=0).mean()

def parsim_trans(x_initial, batch_size, transmission_cap, storage_cap, generation_cap):
    penalties, _, _, _ = sim_trans(x_initial, transmission_cap, batch_size, cf_ts, dem_ts)
    return pd.concat(penalties).sum(axis=0).mean()

def parsim_stor(x_initial, batch_size, transmission_cap, storage_cap, generation_cap):
    penalties, _, _, _ = sim_storage(x_initial, storage_cap, generation_cap, generation_cap, batch_size, cf_ts, dem_ts, eff_c, eff_d)
    return pd.concat(penalties).sum(axis=0).mean()

def parsim_full_flex(x_initial, batch_size, transmission_cap, storage_cap, generation_cap):
    penalties, _, _, _, _, _ = sim_full_flex(x_initial, transmission_cap, storage_cap, generation_cap, generation_cap, batch_size, cf_ts, dem_ts, eff_c, eff_d)
    return pd.concat(penalties).sum(axis=0).mean()

# Simulation

In [ ]:
# Main simulation set-up
transmission_caps = [transmission_cap]
storage_caps = [storage_cap]
generation_caps = [generation_cap] 

scenarios = list(product(transmission_caps, storage_caps, generation_caps))
scenario_names = ["standard"]

In [ ]:
# Run main simulation

for scen, name in zip(scenarios, scenario_names):
    # Check for the following capacities:
    no_cap = [25.*(130+i) for i in range(110)] # 3250-6000
    so_cap = [50.*(37+i) for i in range(163)] # 1850 - 10000
    caps = [np.array(c) for c in list(product(no_cap, so_cap))]
    capacities = list(product(caps,[batch_size], [scen[0]], [scen[1]], [scen[2]]))

    # Run simulations
    with Pool(solvers) as pool:
        resultsim_no = pool.starmap(parsim_no_flex, capacities)
    with Pool(solvers) as pool:
        resultsim_t = pool.starmap(parsim_trans, capacities)
    with Pool(solvers) as pool:
        resultsim_s = pool.starmap(parsim_stor, capacities)
    with Pool(solvers) as pool:
        resultsim_f = pool.starmap(parsim_full_flex, capacities)

    surface_vals = pd.DataFrame(caps, columns = ["NO-N", "NO-S"])
    for res, scen in zip([resultsim_no, resultsim_t, resultsim_s, resultsim_f], ["no-flex", "trans", "stor", "full-flex"]):
        surface_vals[scen] = res
    rel_surface = copy.deepcopy(surface_vals)
    for c in reversed(["no-flex", "trans", "stor", "full-flex"]):
        rel_surface[c] = surface_vals[c]/surface_vals["no-flex"]

    surface_vals.to_csv(f"./results/{name}.csv")

    fig, ax = plt.subplots(figsize=(15,15),subplot_kw={"projection": "3d"})

    X = rel_surface["NO-N"]
    Y = rel_surface["NO-S"]

    colours = ["grey", "green", "orange", "blue"]
    cmaps = ["cividis", "YlGn", "OrRd", "PuBu"]
    for scen, col, maps in zip(["no-flex","trans", "stor", "full-flex",], colours, cmaps):
        Z = rel_surface[scen]
        ax.scatter3D(X,Y,Z, color=col, label = scen)
        ax.plot_trisurf(X,Y,Z, cmap=maps, alpha=0.8)
    ax.set_title(f"Relative penalty depending on capacities, {name[0]}_{name[1]}_{name[2]}")
    ax.set_xlabel("Capacities in NO-N")
    ax.set_ylabel("Capacities in NO-S")

    fig, ax = plt.subplots(figsize=(15,15),subplot_kw={"projection": "3d"})

    X = surface_vals["NO-N"]
    Y = surface_vals["NO-S"]

    colours = ["grey", "green", "orange", "blue"]
    cmaps = ["cividis", "YlGn", "OrRd", "PuBu"]
    for scen, col, maps in zip(["no-flex","trans", "stor", "full-flex",], colours, cmaps):
        Z = surface_vals[scen]
        ax.scatter3D(X,Y,Z, color=col, label = scen)
        ax.plot_trisurf(X,Y,Z, cmap=maps, alpha=0.8)
    ax.set_title(f"Penalty values depending on capacities, {name[0]}_{name[1]}_{name[2]}")
    ax.set_xlabel("Capacities in NO-N")
    ax.set_ylabel("Capacities in NO-S")

# Sensitivity analysis

In [ ]:
# Sensitivity analysis set-up
transmission_caps = [factor * 900. for factor in [0.5, 0.9, 1.1, 1.5]]
storage_caps = [0.5*storage_cap, 0.9 * storage_cap, 1.1 * storage_cap, 1.5 * storage_cap]
generation_caps = [0.5*generation_cap, 0.9*generation_cap, 1.1*generation_cap, 1.5*generation_cap, np.array([450.,900.]), np.array([900., 450.]), np.array([1350., 900.]), np.array([900., 1350.])]

In [ ]:
# Transmission sensitivity analysis
scenario_names = [f"trans{factor}" for factor in [0.5, 0.9, 1.1, 1.5]]
scenarios = list(product(transmission_caps, [storage_cap], [generation_cap]))

In [ ]:
# Sensitivity analysis runs
for scen, name in zip(scenarios, scenario_names):
    # Check for the following capacities:
    no_cap = [50.*(64+i) for i in range(59)] # 3200-6000
    so_cap = [100.*(18+i) for i in range(82)] # 1800 - 10000
    caps = [np.array(c) for c in list(product(no_cap, so_cap))]
    capacities = list(product(caps,[batch_size], [scen[0]], [scen[1]], [scen[2]]))

    # Run simulations
    with Pool(solvers) as pool:
        resultsim_no = pool.starmap(parsim_no_flex, capacities)
    with Pool(solvers) as pool:
        resultsim_t = pool.starmap(parsim_trans, capacities)
    with Pool(solvers) as pool:
        resultsim_s = pool.starmap(parsim_stor, capacities)
    with Pool(solvers) as pool:
        resultsim_f = pool.starmap(parsim_full_flex, capacities)

    surface_vals = pd.DataFrame(caps, columns = ["NO-N", "NO-S"])
    for res, scen in zip([resultsim_no, resultsim_t, resultsim_s, resultsim_f], ["no-flex", "trans", "stor", "full-flex"]):
        surface_vals[scen] = res
    rel_surface = copy.deepcopy(surface_vals)
    for c in reversed(["no-flex", "trans", "stor", "full-flex"]):
        rel_surface[c] = surface_vals[c]/surface_vals["no-flex"]

    surface_vals.to_csv(f"./results/sensitivity_analysis/{name}.csv")

In [ ]:
# Storage sensitivity analysis
scenario_names = [f"storage{factor}" for factor in [0.5, 0.9, 1.1, 1.5]]
scenarios = list(product([transmission_cap], storage_caps, [generation_cap]))

In [ ]:
# Sensitivity analysis runs
for scen, name in zip(scenarios, scenario_names):
    # Check for the following capacities:
    no_cap = [50.*(64+i) for i in range(59)] # 3200-6000
    so_cap = [100.*(18+i) for i in range(82)] # 1800 - 10000
    caps = [np.array(c) for c in list(product(no_cap, so_cap))]
    capacities = list(product(caps,[batch_size], [scen[0]], [scen[1]], [scen[2]]))

    # Run simulations
    with Pool(solvers) as pool:
        resultsim_no = pool.starmap(parsim_no_flex, capacities)
    with Pool(solvers) as pool:
        resultsim_t = pool.starmap(parsim_trans, capacities)
    with Pool(solvers) as pool:
        resultsim_s = pool.starmap(parsim_stor, capacities)
    with Pool(solvers) as pool:
        resultsim_f = pool.starmap(parsim_full_flex, capacities)

    surface_vals = pd.DataFrame(caps, columns = ["NO-N", "NO-S"])
    for res, scen in zip([resultsim_no, resultsim_t, resultsim_s, resultsim_f], ["no-flex", "trans", "stor", "full-flex"]):
        surface_vals[scen] = res
    rel_surface = copy.deepcopy(surface_vals)
    for c in reversed(["no-flex", "trans", "stor", "full-flex"]):
        rel_surface[c] = surface_vals[c]/surface_vals["no-flex"]

    surface_vals.to_csv(f"./results/sensitivity_analysis/{name}.csv")

In [ ]:
# Generation sensitivity analysis
scenario_names = ["gen_0.5", "gen_0.9", "gen_1.1", "gen_1.5", "gen_N0.5", "gen_S0.5", "gen_N1.5", "gen_S1.5"]
scenarios = list(product([transmission_cap], [storage_cap], generation_caps))

In [ ]:

for scen, name in zip(scenarios, scenario_names):
    # Check for the following capacities:
    no_cap = [50.*(64+i) for i in range(59)] # 3200-6000
    so_cap = [100.*(18+i) for i in range(82)] # 1800 - 10000
    caps = [np.array(c) for c in list(product(no_cap, so_cap))]
    capacities = list(product(caps,[batch_size], [scen[0]], [scen[1]], [scen[2]]))

    # Run simulations
    with Pool(solvers) as pool:
        resultsim_no = pool.starmap(parsim_no_flex, capacities)
    with Pool(solvers) as pool:
        resultsim_t = pool.starmap(parsim_trans, capacities)
    with Pool(solvers) as pool:
        resultsim_s = pool.starmap(parsim_stor, capacities)
    with Pool(solvers) as pool:
        resultsim_f = pool.starmap(parsim_full_flex, capacities)

    surface_vals = pd.DataFrame(caps, columns = ["NO-N", "NO-S"])
    for res, scen in zip([resultsim_no, resultsim_t, resultsim_s, resultsim_f], ["no-flex", "trans", "stor", "full-flex"]):
        surface_vals[scen] = res
    rel_surface = copy.deepcopy(surface_vals)
    for c in reversed(["no-flex", "trans", "stor", "full-flex"]):
        rel_surface[c] = surface_vals[c]/surface_vals["no-flex"]

    surface_vals.to_csv(f"./results/sensitvity_analysis/{name}.csv")